In [81]:
import re
import pandas as pd
file_format = "share13"
data_format = "train"
ans = pd.read_csv(r"D:\hw\W2NER-final\acl\data\share13\train.ann", names=["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, na_values=['NaN', 'null'], delimiter='\t')
f_name = open("D:/hw/W2NER-final/answer/"+file_format+"/id/"+data_format+".id","r").read().split("\n")
# pattern = r'[\w]+|[^\s\w]'
pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
all_split_start_end=[]
for i,elmt in  ans.iterrows():
    start_end = elmt['start_end'].split(",")
    entity = elmt['entity']
    split_start_end = []
    word_start = 0
    for j in range(0,len(start_end),2):
        span = [int(s) for s in start_end[j:j+2]]
        start = 0
        sub_entity= entity[word_start:word_start+span[1]-span[0]]
        word_start+=span[1]-span[0]+1
        backend_start = span[0]
        for att in list(re.finditer(pattern_s,sub_entity)):
            sub_span = att.span()
            if att.group()!=" ":
                split_start_end.extend([str(start+backend_start),str(start+backend_start+sub_span[1]-sub_span[0])])
            start += sub_span[1]-sub_span[0]
    all_split_start_end.append(",".join(split_start_end))
    
ans['start_end_split'] = all_split_start_end
ans = ans.rename(columns={"start_end_split":"start_end","start_end":"start_end_old"})

In [78]:
import pandas as pd
from operator import itemgetter
import csv, re
pattern = r'[\w]+|[^\s\w]'
pattern_s = r'[\w]+|\n|[\s]+|[^\s\w]'
ans = pd.read_csv(r"D:\hw\W2NER-final\acl\data\share13\train.ann", names=
                  ["file_id", "label", "start_end", "entity", "other"], keep_default_na=False, quoting=csv.QUOTE_NONE, na_values=['NaN', 'null'], delimiter='\t')
ans_tmp = ans.groupby(['file_id'])
file_format = "share13"
data_format = "train"
f_name = open("D:/hw/W2NER-final/answer/"+file_format+"/id/"+data_format+".id","r").read().split("\n")
def index_label(test_list):
    pre_text = test_list[-1]
    newList = []
    textList = []
    start = 0
    for i in range(len(pre_text)):
        start =  test_list[i][0]
        p_t_s = re.findall(pattern_s,pre_text[i])
        for it,t in enumerate(p_t_s):        
            end = start + len(t)
            if t != ' ':
                textList.append(t)
                newList.append([start, end])
            start = end
    newList.append(textList)
    return newList
all_items_filename_list = []
all_items_type_list = []
all_items_index_list = []
all_items_entity_list = []
for filename in f_name:
    # print(folder_path)
        # if file_format=="cadec":
        #     file_path = folder_path+"/"+filename +".txt"
        # else:
        #     file_path = folder_path+"/"+filename
    true_answers = ans_tmp.get_group(filename)
    target_start_end = true_answers[['start_end']]
    target_entity = true_answers[['entity']]
    index_to_int = []
    target_entity_list = target_entity.values.tolist()
    target_label = true_answers[['label']].values.tolist()
    for s_e_index, s_e in enumerate(target_start_end.values.tolist()):
        s_e = [int(span) for span in s_e[0].split(',')]
        index_to_int.append([s_e, target_entity_list[s_e_index][0],target_label[s_e_index][0]])
    index_to_int = sorted(index_to_int,key=itemgetter(0))
    items_list = []
    item_s_e = {}
    for item in index_to_int:
        items = []
        if len(item[0])>2:
            start = 0
            item_str = []
            item_index = []
            itm = item[1]
            for index in range(0, len(item[0]), 2):     
                end = start + (item[0][index+1] - item[0][index])
                item_str.append(itm[start:end])
                itm = itm[end:].strip()
                item_index.append([item[0][index], item[0][index+1]])
            items = item_index+[item_str]
        else:
            items = [item[0], [item[1]]]
        
        newList = index_label(items)
        
        positions = ",".join([str(item) for sublist in newList[:-1] for item in sublist])
        all_items_filename_list.append(filename)
        all_items_type_list.append(item[2])
        all_items_index_list.append(positions)
        all_items_entity_list.append(",".join(newList[-1]))
   
# break
    # all_items_list.extend(items_list)

In [79]:
dict_df = {
    "file_id":all_items_filename_list,
    "label":all_items_type_list,
    "start_end":all_items_index_list,
    "entity":all_items_entity_list
}
df = pd.DataFrame(dict_df)
df

,file_id,label,start_end,entity
0,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"330,338",headache
1,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"343,352",dizziness
2,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"392,400",headache
3,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"406,410,411,420","neck,stiffness"
4,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"429,435,436,438,439,443","unable,to,walk"
...,...,...,...,...
5146,23893-094803-ECG_REPORT.txt,Disorder,"165,176,177,183","ventricular,ectopy"
5147,23893-094803-ECG_REPORT.txt,Disorder,"238,243,244,250","sinus,rhythm"
5148,23893-094803-ECG_REPORT.txt,Disorder,"269,275,276,282","atrial,ectopy"
5149,23893-094803-ECG_REPORT.txt,Disorder,"287,299,300,306","ventriciular,ectopy"


In [85]:
ans.merge(df,on=["file_id","start_end"])

,file_id,label_x,start_end_old,entity_x,other,start_end,label_y,entity_y
0,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"330,338",headache,,"330,338",Disorder,headache
1,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"1141,1148,1192,1198",Abdomen bruits,"is soft, nontender, nondistended, negative","1141,1148,1192,1198",Disorder,"Abdomen,bruits"
2,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"1940,1952",MCA aneurysm,,"1940,1943,1944,1952",Disorder,"MCA,aneurysm"
3,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"658,672",Hypothyroidism,,"658,672",Disorder,Hypothyroidism
4,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"725,729",rash,,"725,729",Disorder,rash
...,...,...,...,...,...,...,...,...
5146,25844-097135-ECHO_REPORT.txt,Disorder,"859,861",MR,,"859,861",Disorder,MR
5147,25844-097135-ECHO_REPORT.txt,Disorder,"927,929",TR,,"927,929",Disorder,TR
5148,25844-097135-ECHO_REPORT.txt,Disorder,"949,969",pericardial effusion,,"949,960,961,969",Disorder,"pericardial,effusion"
5149,25844-097135-ECHO_REPORT.txt,Disorder,"994,1005,1016,1023",left atrium dilated,is mildly,"994,998,999,1005,1016,1023",Disorder,"left,atrium,dilated"


In [72]:
ans.insert(2, column="engilsh", value=all_items_index_list) 

ValueError: Length of values (5151) does not match length of index (5821)

,file_id,label,start_end,entity,other
0,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"330,338",headache,
1,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"1141,1148,1192,1198",Abdomen bruits,"is soft, nontender, nondistended, negative"
2,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"1940,1952",MCA aneurysm,
3,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"658,672",Hypothyroidism,
4,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"725,729",rash,
...,...,...,...,...,...
5816,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1060,1093",pulmonary vascular redistribution,
5817,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1154,1170",pleural effusion,
5818,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1276,1288",pneumothorax,
5819,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1330,1333",CHF,


In [68]:
a = [[406, 410], [411, 420], ['neck', 'stiffness']]
# for i in a[:-1]:
#     for j in i:
#         print(j)
# aa = [print(j) for i in a[:-1] for j in i]
# aa
positions = ",".join([str(item) for sublist in a[:-1] for item in sublist])
aa = ",".join(a[-1])
print(positions)
print(aa)



406,410,411,420
neck,stiffness


In [29]:
all_items_list[4][:-2]

[[429, 435], [436, 438], [439, 443]]

In [31]:
a = [i for i in index[:-2] for index in all_items_list]
a[3]

[[406, 410], [411, 420]]

In [26]:
all_items_list

[[[330, 338], ['headache'], 'Disorder'],
 [[343, 352], ['dizziness'], 'Disorder'],
 [[392, 400], ['headache'], 'Disorder'],
 [[406, 410], [411, 420], ['neck', 'stiffness'], 'Disorder'],
 [[429, 435], [436, 438], [439, 443], ['unable', 'to', 'walk'], 'Disorder'],
 [[536, 545], [546, 548], [549, 553], ['inability', 'to', 'walk'], 'Disorder'],
 [[562, 566], ['pain'], 'Disorder'],
 [[579, 587], ['headache'], 'Disorder'],
 [[658, 672], ['Hypothyroidism'], 'Disorder'],
 [[725, 729], ['rash'], 'Disorder'],
 [[888, 893], [894, 902], ['acute', 'distress'], 'Disorder'],
 [[1141, 1148], [1158, 1167], ['Abdomen', 'nontender'], 'Disorder'],
 [[1141, 1148], [1192, 1198], ['Abdomen', 'bruits'], 'Disorder'],
 [[1169, 1181], ['nondistended'], 'Disorder'],
 [[1218, 1226], ['clubbing'], 'Disorder'],
 [[1228, 1236], ['cyanosis'], 'Disorder'],
 [[1241, 1246], ['edema'], 'Disorder'],
 [[1327, 1335], [1336, 1344], ['cerebral', 'aneurysm'], 'Disorder'],
 [[1389, 1392], [1412, 1420], ['MCA', 'aneurysm'], 'Diso

In [21]:
[[[330, 338], ['headache'], 'Disorder'], 
 [[343, 352], ['dizziness'], 'Disorder'], 
 [[392, 400], ['headache'], 'Disorder'], 
 [[406, 410], [411, 420], ['neck', 'stiffness'], 'Disorder'], 
 [[429, 435], [436, 438], [439, 443], ['unable', 'to', 'walk'], 'Disorder'], 
 [[536, 545], [546, 548], [549, 553], ['inability', 'to', 'walk'], 'Disorder'], 
 [[562, 566], ['pain'], 'Disorder'], [[579, 587], ['headache'], 'Disorder'], 
 [[658, 672], ['Hypothyroidism'], 'Disorder'], [[725, 729], ['rash'], 'Disorder'], 
 [[888, 893], [894, 902], ['acute', 'distress'], 'Disorder'], [[1141, 1148], [1158, 1167], ['Abdomen', 'nontender'], 'Disorder'], 
 [[1141, 1148], [1192, 1198], ['Abdomen', 'bruits'], 'Disorder'], [[1169, 1181], ['nondistended'], 'Disorder'], [[1218, 1226], ['clubbing'], 'Disorder'], [[1228, 1236], ['cyanosis'], 'Disorder'], [[1241, 1246], ['edema'], 'Disorder'], [[1327, 1335], [1336, 1344], ['cerebral', 'aneurysm'], 'Disorder'], [[1389, 1392], [1412, 1420], ['MCA', 'aneurysm'], 'Disorder'], [[1552, 1560], ['afebrile'], 'Disorder'], [[1654, 1660], [1661, 1666], ['facial', 'droop'], 'Disorder'], [[1671, 1676], ['drift'], 'Disorder'], [[1852, 1860], ['aneurysm'], 'Disorder'], [[1918, 1926], ['aneurysm'], 'Disorder'], [[1940, 1943], [1944, 1952], ['MCA', 'aneurysm'], 'Disorder'], [[2150, 2153], [2154, 2162], ['MCA', 'aneurysm'], 'Disorder'], [[2319, 2327], ['headache'], 'Disorder'], [[2329, 2335], [2336, 2339], [2340, 2351], ['moving', 'all', 'extremities'], 'Disorder'], 
 [[2374, 2379], ['drift'], 'Disorder']]
    

In [ ]:
df.insert(2, column="engilsh", value=[88, 72, 74, 98])
print("在第三欄的地方新增一個欄位資料")
print(df)

In [22]:
items_list

[[[95, 100], [101, 107], ['Sinus', 'rhythm'], 'Disorder'],
 [[109, 113],
  [114, 120],
  [121, 132],
  ['Left', 'atrial', 'abnormality'],
  'Disorder'],
 [[134, 140], [141, 149], ['Atrial', 'bigeminy'], 'Disorder'],
 [[165, 176], [177, 183], ['ventricular', 'ectopy'], 'Disorder'],
 [[238, 243], [244, 250], ['sinus', 'rhythm'], 'Disorder'],
 [[269, 275], [276, 282], ['atrial', 'ectopy'], 'Disorder'],
 [[287, 299], [300, 306], ['ventriciular', 'ectopy'], 'Disorder'],
 [[325, 329], ['rate'], 'Disorder']]

In [18]:
index_to_int

[[[95, 107], 'Sinus rhythm', 'Disorder'],
 [[109, 132], 'Left atrial abnormality', 'Disorder'],
 [[134, 149], 'Atrial bigeminy', 'Disorder'],
 [[165, 183], 'ventricular ectopy', 'Disorder'],
 [[238, 250], 'sinus rhythm', 'Disorder'],
 [[269, 282], 'atrial ectopy', 'Disorder'],
 [[287, 306], 'ventriciular ectopy', 'Disorder'],
 [[325, 329], 'rate', 'Disorder']]

In [ ]:
index_to_int = []
target_entity_list = target_entity.values.tolist()
target_label = true_answers[['label']].values.tolist()
for s_e_index, s_e in enumerate(target_start_end.values.tolist()):
    s_e = [int(span) for span in s_e[0].split(',')]
    index_to_int.append([s_e, target_entity_list[s_e_index][0],target_label[s_e_index][0]])
# index_to_int = sorted(index_to_int,key=itemgetter(0))

# items_list = []
# item_s_e = {}
# for item in index_to_int:
#     items = []
#     # print(item)
#     if len(item[0])>2:
#         start = 0
#         item_str = []
#         item_index = []
#         itm = item[1]
#         for index in range(0, len(item[0]), 2):     
#             end = start + (item[0][index+1] - item[0][index])
#             item_str.append(itm[start:end])
#             itm = itm[end:].strip()
#             item_index.append([item[0][index], item[0][index+1]])
#         items = item_index+[item_str]
#     else:
#         items = [item[0], [item[1]]]
#     # print(items)
#     newList = index_label(items)+[item[2]]

In [12]:
target_entity

,entity
5525,Sinus rhythm
5526,ventricular ectopy
5527,sinus rhythm
5528,atrial ectopy
5529,ventriciular ectopy
5530,rate
5531,Left atrial abnormality
5532,Atrial bigeminy


In [8]:
true_answers

,file_id,label,start_end,entity,other
5525,23893-094803-ECG_REPORT.txt,Disorder,"95,107",Sinus rhythm,
5526,23893-094803-ECG_REPORT.txt,Disorder,"165,183",ventricular ectopy,
5527,23893-094803-ECG_REPORT.txt,Disorder,"238,250",sinus rhythm,
5528,23893-094803-ECG_REPORT.txt,Disorder,"269,282",atrial ectopy,
5529,23893-094803-ECG_REPORT.txt,Disorder,"287,306",ventriciular ectopy,
5530,23893-094803-ECG_REPORT.txt,Disorder,"325,329",rate,
5531,23893-094803-ECG_REPORT.txt,Disorder,"109,132",Left atrial abnormality,
5532,23893-094803-ECG_REPORT.txt,Disorder,"134,149",Atrial bigeminy,


In [3]:
true_answers

,file_id,label,start_end,entity,other
0,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"330,338",headache,
1,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"1141,1148,1192,1198",Abdomen bruits,"is soft, nontender, nondistended, negative"
2,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"1940,1952",MCA aneurysm,
3,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"658,672",Hypothyroidism,
4,00098-016139-DISCHARGE_SUMMARY.txt,Disorder,"725,729",rash,
...,...,...,...,...,...
5816,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1060,1093",pulmonary vascular redistribution,
5817,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1154,1170",pleural effusion,
5818,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1276,1288",pneumothorax,
5819,26563-387055-RADIOLOGY_REPORT.txt,Disorder,"1330,1333",CHF,
